In [ ]:
!pip install transformers

In [ ]:
from pandas import read_csv

In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
def calc_sentiment(input):
  # id2label = {0: "negative", 1: "neutral", 2: "positive"}
  tokenizer = AutoTokenizer.from_pretrained("Voicelab/herbert-base-cased-sentiment")
  model = AutoModelForSequenceClassification.from_pretrained("Voicelab/herbert-base-cased-sentiment").to("cuda:0")
  encoding = tokenizer(
            input,
            add_special_tokens=True,
            return_token_type_ids=True,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
          ).to("cuda:0")
  output = model(**encoding).logits.to("cuda:0").detach().cpu().numpy()
  # prediction = id2label[np.argmax(output)]
  prediction = np.argmax(output)
  return prediction

In [ ]:
import time

class Timer:
    _start_time = None
    @classmethod
    def start_measure(cls):
        cls._start_time = time.perf_counter()

    @classmethod
    def get_time(cls):
        return time.perf_counter() - cls._start_time

    @classmethod
    def print_time_elapsed(cls):
        print('Time elapsed: {:.3f} s'.format(cls.get_time()))


In [ ]:
df = read_csv("database.csv", lineterminator='\n')
# Ignore N/A subjects
df_filtered = df[df['subjects'].notna()]

In [ ]:
df_filtered['sentiment'] = df_filtered['summary'].apply(calc_sentiment)
df_filtered.to_csv('df_out_sample.csv')

Try on dataframe small parts

In [ ]:
df_sample = df.loc[:10]
df_sample_30 = df.loc[:30]

In [ ]:
Timer.start_measure()
df_sample['sentiment'] = df_sample['summary'].apply(calc_sentiment)
Timer.print_time_elapsed()

In [ ]:
Timer.start_measure()
df_sample['sentiment'] = df_sample['summary'].apply(calc_sentiment)
Timer.print_time_elapsed()

Timer.start_measure()
df_sample['sentiment_title'] = df_sample['title'].apply(calc_sentiment)
Timer.print_time_elapsed()

Timer.start_measure()
df_sample_30['sentiment'] = df_sample_30['summary'].apply(calc_sentiment)
Timer.print_time_elapsed()

Timer.start_measure()
df_sample_30['sentiment_title'] = df_sample_30['title'].apply(calc_sentiment)
Timer.print_time_elapsed()